In [100]:
#https://www.kaggle.com/c/nlp-getting-started/data
import pandas as pd
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import string
import matplotlib.pyplot as plt
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
stop_words = set(stopwords.words('english'))
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gauravtyagi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gauravtyagi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gauravtyagi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [101]:
test= pd.read_csv("/Users/gauravtyagi/Downloads/Data/nlp-getting-started/test.csv")
train= pd.read_csv("/Users/gauravtyagi/Downloads/Data/nlp-getting-started/train.csv")

In [102]:
test.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [103]:
def encode_data(data,col) :
    label_encoders = {}  
    print("Encoding {}".format(col))
    new_le = LabelEncoder()
    data[col] = new_le.fit_transform(data[col])
    return data 

In [104]:
def preprocess_data(data) :
    data['location']=data['location'].fillna('')
    data['keyword']=data['keyword'].fillna('')
    return data

In [105]:
train = preprocess_data(train)
test = preprocess_data(test)

In [106]:
train['keyword'].value_counts()

                         61
fatalities               45
armageddon               42
deluge                   42
harm                     41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 222, dtype: int64

In [107]:
train['location'].value_counts()

                              2533
USA                            104
New York                        71
United States                   50
London                          45
                              ... 
LA                               1
Host of #MindMoversPodcast       1
BiÌ±an,Laguna                    1
IRAQ                             1
 Neverland                       1
Name: location, Length: 3342, dtype: int64

In [108]:
train['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [109]:
encode_data(train,'location')

Encoding location


,id,keyword,location,text,target
0,1,,0,Our Deeds are the Reason of this #earthquake M...,1
1,4,,0,Forest fire near La Ronge Sask. Canada,1
2,5,,0,All residents asked to 'shelter in place' are ...,1
3,6,,0,"13,000 people receive #wildfires evacuation or...",1
4,7,,0,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,,0,Two giant cranes holding a bridge collapse int...,1
7609,10870,,0,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,,0,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,,0,Police investigating after an e-bike collided ...,1


In [110]:
train['location'].value_counts()

0       2533
2644     104
1827      71
2663      50
1507      45
        ... 
2971       1
922        1
2967       1
918        1
2041       1
Name: location, Length: 3342, dtype: int64

In [67]:
def process_data(tweet):
    
    #make tweet in lower cases 
    tweet = tweet.lower()
    
    # remove urls
    tweet= re.sub(r"http\S+|www\S+|https\S+","",tweet,flags=re.MULTILINE)
    
    # remove punctuation 
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    
    # remove user @ and #
    tweet = re.sub(r'\@\w+|\#',"",tweet)
    
    # remove stopwords
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [word for word in tweet_tokens if word not in stop_words]

    # Stemming 
    ps = PorterStemmer()
    stemmed_word = [ps.stem(w) for w in filtered_words]
    #print('stemmed_word ' +  " ".join(stemmed_word))

    # lemmatizing 
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w , pos='a') for w in stemmed_word]

    return " ".join(lemma_words)

In [68]:
process_data("Just happened a terrible car crash")

'happen terribl car crash'

In [69]:
train['text'] = train['text'].apply(process_data)  

In [70]:
test['text'] = test['text'].apply(process_data)  

In [71]:
def get_vector(train) :
    vector = TfidfVectorizer(sublinear_tf= True)
    vector.fit(train)
    return vector

In [76]:
#Function to ngram
def get_top_n_gram(corpus,ngram_range,n=None):
    vec = CountVectorizer(ngram_range=ngram_range,stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]


In [77]:
#n2_bigram
train_bigrams = get_top_n_gram(train['text'],(2,2),20)
train_bigrams

[('bodi bag', 76),
 ('mass murder', 61),
 ('suicid bomber', 60),
 ('burn build', 58),
 ('look like', 55),
 ('atom bomb', 51),
 ('train derail', 50),
 ('gon na', 43),
 ('youtub video', 43),
 ('like youtub', 42),
 ('northern california', 41),
 ('cross bodi', 39),
 ('oil spill', 39),
 ('suicid bomb', 38),
 ('california wildfir', 36),
 ('wan na', 35),
 ('feel like', 32),
 ('70 year', 31),
 ('bomber deton', 31),
 ('natur disast', 30)]

In [78]:
from sklearn.linear_model import LogisticRegression
x_train, x_test, y_train, y_test = train_test_split(train['text'], train['target'], test_size=0.25, random_state=0)

In [79]:
x_train.head(5)

5244    refugio oil spill may costly big project plain...
4860    julian knight scvsupremecourt dismiss mass mur...
6538    electr cant stop scofield nigga surviv hotbox ...
5175    meek mill beg nicki minaj let obliter ovofest ...
5820    chinaûª stock market crash summer spark inter...
Name: text, dtype: object

In [80]:
logisticRegr = LogisticRegression()

In [81]:
logisticRegr.fit(x_train, y_train)

ValueError: could not convert string to float: 'refugio oil spill may costly big project plain american pipelin oil spill'